### Introducation

##### What are variational algorithms?

The available quantum computers in the near term, called Noisy Intermediate Scale Quantum computers (NISQ), are too small to support error correction and have high levels of noise. Therefore, implementing long quantum circuits on such devices will not likely provide satisfyingly accurate results. A class of algorithms, called the variational algorithms, on the other hand are suited to be performed on a quantum computer since they can be implemented using shallow circuits.

One of these variational algorithms called VQE.

The Variational Quantum Eigensolver (VQE) is a flagship algorithm for quantum chemistry using near-term quantum computers.
It is an application of the Ritz variational principle, where a quantum computer is trained to prepare the ground state of a given molecule. This algorithm is a Hybrid algorithm as it divides the task of finding the smallest eigenvalue between quantum and classical processors. This is done by taking an advantage of the natural preparation of variational quantum states and measuring expectation values on quantum computers, which are difficult to do on classical computers due to the expoenential growth of the Hilbert space (quantum qubits state space) with system size (the number of qubits). Moreover, the algorithm uses classical processors to do tasks they are capable of performing easily. More specifically, VQE uses a classical optimizer to minimize the expectation value obtained from the quantum computer by varying the variational qubits state.

The inputs to the VQE algorithm are a Hamiltonian (mulacular or modeled) and a parametrized circuit preparing the quantum state of the molecule. Within VQE, the cost function is defined as the expectation value of the Hamiltonian computed in the trial state. The ground state of the target Hamiltonian is obtained by performing an iterative minimization of the cost function. The optimization is carried out by a classical optimizer which leverages a quantum computer to evaluate the cost function and calculate its gradient at each optimization step.

### Basic Imports

In [22]:
from qiskit.circuit.library.standard_gates import RXGate, RZGate, RYGate, CXGate, CZGate, SGate, HGate
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from typing import Dict, Tuple, List
import numpy as np
from numpy import linalg as LA
from qiskit.opflow import X, Z, I, H, Y
from qiskit.quantum_info import Pauli
import matplotlib
from qiskit import assemble, Aer
from qiskit import IBMQ
from qiskit import *
from qiskit.visualization import plot_histogram
import math
from qiskit.algorithms.optimizers import L_BFGS_B, COBYLA

### full entengalment and linear entanglment ansatz circuits

for information about the VQE ansatz circuits, see the relevent notenook in this folder.

In [23]:
import nbimporter
from linear_entangelment_and_full_entangelment_ansatz_circuits import *

# Generating the hamiltonians

for information about the hamitonians, see the relevent notenook in this folder.

before each iteration on specific hamitonian, the hamiltonian matrix will be printed.

In [24]:
from hamiltonians import *

# Expectation Value

In [25]:
reducing_to_pauli_z_dict = {
    Pauli('I'): Pauli('I'),
    Pauli('Z'): Pauli('Z'),
    Pauli('X'): Pauli('Z'),
    Pauli('Y'): Pauli('Z')
} 

def transfrom_hamiltonian_into_pauli_string(hamiltonian):
    pauli_operators = hamiltonian.to_pauli_op().settings['oplist']
    pauli_strings = list(map(lambda pauli_operator: pauli_operator.primitive, pauli_operators))
    pauli_coeffs = list(map(lambda pauli_operator: pauli_operator.coeff, pauli_operators))
    return (pauli_strings, pauli_coeffs)

def reduce_pauli_matrixes_into_sigma_z(pauli_string):
    for matrix_index in range(QUBITS_NUM):
        pauli_matrix = pauli_string[matrix_index]
        pauli_string[matrix_index].insert(reducing_to_pauli_z_dict[pauli_matrix])
    
    return pauli_string

def get_z_reduction_for_pauli_string(qc, pauli_string):
    qr = QuantumRegister(QUBITS_NUM, name="q")
    exdend_qc = QuantumCircuit(qr)
    pauli_string = str(pauli_string)
    for qubit_index, pauli_matrix in enumerate(pauli_string):
        if pauli_matrix == "X":
            exdend_qc.append(HGate(), [qr[qubit_index]])
        elif pauli_matrix == "Y":
            exdend_qc.append(HGate(), [qr[qubit_index]])
            exdend_qc.append(SGate(), [qr[qubit_index]])
    qc = qc.compose(exdend_qc)
    return qc

def get_probability_distribution(counts: Dict) -> Dict:
    proba_distribution = {state: (count / NUM_SHOTS) for state, count in counts.items()}
    return proba_distribution

def calculate_probabilities_of_measurments_in_computational_basis(quantum_state_circuit) -> Dict:
    quantum_state_circuit.measure_all()

    transpiled_quantum_state_circuit = transpile(quantum_state_circuit, BACKEND) 
    
    result = BACKEND.run(transpiled_quantum_state_circuit).result() 
    counts = result.get_counts(transpiled_quantum_state_circuit)
    
    return get_probability_distribution(counts)

def sort_probas_dict_by_qubits_string_keys(proba_distribution: Dict) -> Dict:
    return dict(sorted(proba_distribution.items()))

def reset_power_of_minus_1(power_of_minus_1):
    power_of_minus_1 = 0
    return power_of_minus_1

def calculate_expectation_value_of_pauli_string_by_measurments_probas(pauli_string, probas_distribution):
    pauli_string_expectation_value = 0
    power_of_minus_1 = 0
    
    sorted_probas_distribuition = sort_probas_dict_by_qubits_string_keys(probas_distribution)
    for qubits_string, proba in sorted_probas_distribuition.items():
        for string_index in range(QUBITS_NUM):
            if(str(qubits_string[string_index])=="1" and str(pauli_string[string_index])=="Z"):
                power_of_minus_1 += 1
            
        pauli_string_expectation_value += pow(-1, power_of_minus_1)*proba
        power_of_minus_1 = reset_power_of_minus_1(power_of_minus_1)
        
    return pauli_string_expectation_value

def get_expectation_value(pauli_string, probas_distribution):
    return calculate_expectation_value_of_pauli_string_by_measurments_probas(pauli_string, probas_distribution)

# Calculating the target function

In [26]:
def target_function(thetas):
    ansatz = calc_ansatz_circ(QUBITS_NUM, Entangelment, thetas, Depth)
    
    qr = QuantumRegister(QUBITS_NUM, name="qubit")
    qc = QuantumCircuit(qr)
    qc.initialize(Init_state, qc.qubits)
    
    total_expectation_value = 0
    pauli_strings, pauli_coeffs = transfrom_hamiltonian_into_pauli_string(Ham)
    
    for pauli_index, pauli_string in enumerate(pauli_strings):
        qc_per_pauli_string = qc.compose(ansatz)
        qc_per_pauli_string = get_z_reduction_for_pauli_string(qc_per_pauli_string, pauli_string)
        probas_distribution = calculate_probabilities_of_measurments_in_computational_basis(qc_per_pauli_string)
        total_expectation_value += pauli_coeffs[pauli_index] * get_expectation_value(pauli_string, probas_distribution)
        
    print("total expectation value:")
    print(total_expectation_value)
    return total_expectation_value
        

# Calculate the difference between the algorithm result and actual result

In [27]:
def calc_diff_from_actual_result(hamiltonian, algorithm_result):
    matrix = hamiltonian.to_matrix()
    eigen_values, eigen_vectors = LA.eig(matrix)
    actual_result = min(eigen_values)
    print("The actual minimal energy is:")
    print(actual_result)
    diff = algorithm_result - actual_result
    print("The differnce between the actual value and the algorithm result is:")
    print(diff)

# Implementing the VQE algorithem

In [28]:
def VQE(optimazer, initial_thetas):
    print("The Hamiltonian running VQE on is:")
    print(Ham)
    point, value, nfev = optimazer.optimize(QUBITS_NUM*2*(Depth + 1), target_function, initial_point=initial_thetas)
    base_energy = value
    print("The VQE result for the minimal energy of the Hamiltonian given is:")
    print(base_energy)
    calc_diff_from_actual_result(Ham, base_energy)

# Setting the simulator and optimazer

In [29]:
bfgs_optimizer = L_BFGS_B(maxiter=2)
cobyla_optimazer = COBYLA(maxiter=50)

simulator_backend = Aer.get_backend('qasm_simulator') 

# VQE with Transverse Ising Model Hamiltonian(4 Qubits) + LINEAR Entangelment Ansatz

In [30]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 4
Ham = get_hamiltonian("Ising Model")
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)
    

NameError: name 'get_hamiltonian' is not defined

# VQE with Transverse Ising Model Hamiltonian(4 Qubits) + FULL Entangelment Ansatz

In [ ]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 4
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)

# VQE with Transverse Ising Model Hamiltonian(4 Qubits) + LINEAR Entangelment Ansatz Using COBYLA Optimazer

In [ ]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 4
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)
    

# VQE with Transverse Ising Model Hamiltonian(4 Qubits) + FULL Entangelment Ansatz Using COBYLA Optimazer

In [ ]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 4
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)

# VQE with Transverse Ising Model Hamiltonian(3 Qubits) + LINEAR Entangelment Ansatz

In [ ]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 3
Ham = get_hamiltonian("Ising Model")
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)
    

# VQE with Transverse Ising Model Hamiltonian(3 Qubits) + FULL Entangelment Ansatz

In [ ]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 3
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)

# VQE with Transverse Ising Model Hamiltonian(3 Qubits) + LINEAR Entangelment Ansatz Using COBYLA Optimazer

In [ ]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 3
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)
    

# VQE with Transverse Ising Model Hamiltonian(3 Qubits) + FULL Entangelment Ansatz Using COBYLA Optimazer

In [ ]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 3
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)

# VQE with Transverse Ising Model Hamiltonian(2 Qubits) + LINEAR Entangelment Ansatz

In [ ]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 2
Ham = get_hamiltonian("Ising Model")
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)
    

# VQE with Transverse Ising Model Hamiltonian(2 Qubits) + FULL Entangelment Ansatz

In [ ]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 2
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)

# VQE with Transverse Ising Model Hamiltonian(2 Qubits) + LINEAR Entangelment Ansatz Using COBYLA Optimazer

In [ ]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 2
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)
    

# VQE with Transverse Ising Model Hamiltonian(2 Qubits) + FULL Entangelment Ansatz Using COBYLA Optimazer

In [ ]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 2
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)

# VQE with H2 Molecule Hamiltonian(2 Qubits) + LINEAR Entangelment Ansatz

In [ ]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 2
Ham = get_hamiltonian("H2")
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)

# VQE with H2 Molecule Hamiltonian(2 Qubits) + FULL Entangelment Ansatz

In [ ]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 2
Ham = get_hamiltonian("H2")
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)

# VQE with H2 Molecule Hamiltonian(2 Qubits) + LINEAR Entangelment Ansatz Using COBYLA Optimazer

In [ ]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 2
Ham = get_hamiltonian("H2")
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)

# VQE with H2 Molecule Hamiltonian(2 Qubits) + FULL Entangelment Ansatz Using COBYLA Optimazer

In [ ]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 2
Ham = get_hamiltonian("H2")
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)

# VQE with H2 Molecule Hamiltonian(4 Qubits) + LINEAR Entangelment Ansatz

In [ ]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 4
Ham = get_hamiltonian("H2")
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)

# VQE with H2 Molecule Hamiltonian(4 Qubits) + FULL Entangelment Ansatz

In [ ]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 4
Ham = get_hamiltonian("H2")
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)

# VQE with H2 Molecule Hamiltonian(4 Qubits) + LINEAR Entangelment Ansatz Using COBYLA Optimazer

In [ ]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 4
Ham = get_hamiltonian("H2")
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)

# VQE with H2 Molecule Hamiltonian(4 Qubits) + FULL Entangelment Ansatz Using COBYLA Optimazer

In [ ]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 4
Ham = get_hamiltonian("H2")
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)

# VQE with LiH Molecule Hamiltonian(4 Qubits) + LINEAR Entangelment Ansatz

In [ ]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 4
Ham = get_hamiltonian("LiH")
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)

# VQE with LiH Molecule Hamiltonian(4 Qubits) + FULL Entangelment Ansatz

In [ ]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 4
Ham = get_hamiltonian("LiH")
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)

# VQE with LiH Molecule Hamiltonian(4 Qubits) + LINEAR Entangelment Ansatz Using COBYLA Optimazer

In [ ]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 4
Ham = get_hamiltonian("LiH")
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)

# VQE with LiH Molecule Hamiltonian(4 Qubits) + FULL Entangelment Ansatz Using COBYLA Optimazer

In [ ]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 4
Ham = get_hamiltonian("LiH")
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)